In [1]:
import math
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.cluster import KMeans
import numpy as numpy

In [8]:
#Data = pd.read_csv("xor.csv")
#Data.head()

Data= pd.read_csv("bank-full.csv")
cols= ["age","balance","day","duration","campaign","pdays","previous"]
data_encode= Data.drop(cols, axis= 1)
data_encode= data_encode.apply(LabelEncoder().fit_transform)
data_rest= Data[cols]
Data= pd.concat([data_rest,data_encode], axis= 1)

In [10]:
data_train, data_test = train_test_split(Data, test_size = 0.5, random_state = 4)
#data_train = Data
#data_test = Data
x_train = data_train.drop("y", axis = 1)
y_train = data_train["y"]
x_test = data_test.drop("y", axis = 1)
y_test = data_test["y"]
#x_train.head()

scaler= StandardScaler()
scaler.fit(x_train)
x_train= scaler.transform(x_train)
x_test= scaler.transform(x_test)


In [11]:
#K_cent = 4
K_cent = 8
km = KMeans(n_clusters = K_cent, max_iter= 100)
km.fit(x_train)
cent = km.cluster_centers_

In [12]:
max = 0
for i in range(K_cent):
    for j in range(K_cent):
        d = numpy.linalg.norm(cent[i] - cent[j])
        if d > max:
            max = d

d = max
sigma = d / math.sqrt(2 * K_cent)
print(sigma)

2.290119075581456


In [13]:
#x_train = x_train.to_numpy()
#print(x_train)

In [14]:
shape = x_train.shape
row = shape[0]
column = K_cent
G = numpy.empty((row, column), dtype = float)
for i in range(row):
    for j in range(column):
        dist = numpy.linalg.norm(x_train[i] - cent[j])
        G[i][j] = math.exp(-math.pow(dist, 2) / math.pow(2 * sigma, 2))
print(G)

[[0.17168699 0.24107285 0.21536531 ... 0.07926195 0.01854559 0.6563449 ]
 [0.46102001 0.80532781 0.45809963 ... 0.22312612 0.04657763 0.30699505]
 [0.46672329 0.65096728 0.52783906 ... 0.31888228 0.04389327 0.35317357]
 ...
 [0.59285971 0.31994197 0.37471284 ... 0.13260172 0.03749151 0.23787698]
 [0.43189788 0.56053299 0.67431727 ... 0.17199614 0.04163052 0.28260542]
 [0.46929082 0.72668184 0.49630998 ... 0.23897415 0.04744916 0.31944676]]


In [15]:
GTG = numpy.dot(G.T, G)
GTG_inv = numpy.linalg.inv(GTG)
fac = numpy.dot(GTG_inv, G.T)
W = numpy.dot(fac, y_train)
print(W)

[-0.08973626  0.14363653 -0.26520059  0.25752932 -0.23307462  0.22410509
  0.08888001  0.23819604]


In [16]:
row = x_test.shape[0]
column = K_cent
G_test = numpy.empty((row, column), dtype = float)
for i in range(row):
    for j in range(column):
        #dist = numpy.linalg.norm(x_test.iloc[i] - cent[j])
        dist = numpy.linalg.norm(x_test[i] - cent[j])
        G_test[i][j] = math.exp(-math.pow(dist, 2) / math.pow(2 * sigma, 2))
print(G_test[0])

[0.4563352  0.74695686 0.47935883 0.59100608 0.35895325 0.17754991
 0.04610366 0.29682798]


In [19]:
prediction = numpy.dot(G_test, W)
prediction = 0.5 * (numpy.sign(prediction - 0.5) + 1)
score = accuracy_score(prediction, y_test)
print(score)

0.8860479518711847
